[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb)

# Dynamic Routes

In semantic-router there are two types of routes that can be chosen. Both routes belong to the `Route` object, the only difference between them is that _static_ routes return a `Route.name` when chosen, whereas _dynamic_ routes use an LLM call to produce parameter input values.

For example, a _static_ route will tell us if a query is talking about mathematics by returning the route name (which could be `"math"` for example). A _dynamic_ route can generate additional values, so it may decide a query is talking about maths, but it can also generate Python code that we can later execute to answer the user's query, this output may look like `"math", "import math; output = math.sqrt(64)`.

***⚠️ Note: We have a fully local version of dynamic routes available at [docs/05-local-execution.ipynb](https://github.com/aurelio-labs/semantic-router/blob/main/docs/05-local-execution.ipynb). The local 05 version tends to outperform the OpenAI version we demo in this notebook, so we'd recommend trying [05](https://github.com/aurelio-labs/semantic-router/blob/main/docs/05-local-execution.ipynb)!***

## Installing the Library

In [9]:
!pip install -qU semantic-router==0.0.32
!pip install unifyai


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\indir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\indir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Initializing Routes and RouteLayer

Dynamic routes are treated in the same way as static routes, let's begin by initializing a `RouteLayer` consisting of static routes.

In [11]:
from semantic_router import Route

# Define routes for Math and Coding
math_route = Route(
    name="math",
    utterances=[
        "solve for x in the equation",
        "what is the integral of",
        "how to calculate the derivative",
        "mathematical proofs",
        "how do you find the percentage of this number"
    ],
)

coding_route = Route(
    name="coding",
    utterances=[
        "how to write a for loop in Python",
        "explain the use of classes in Java",
        "what is recursion in programming",
        "how do i optimise this problem using hash tables",
        "suggest a more efficient data structure for this problem"
    ],
)

# List of all routes
routes = [math_route, coding_route]

We initialize our `RouteLayer` with our `encoder` and `routes`. We can use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`, or local alternatives like `FastEmbedEncoder`.

In [12]:
import os
from getpass import getpass
from semantic_router import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

# dashboard.cohere.ai
# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
#     "Enter Cohere API Key: "
# )
# platform.openai.com
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

# encoder = CohereEncoder()
encoder = OpenAIEncoder()

rl = RouteLayer(encoder=encoder, routes=routes)

2024-04-18 21:19:32 INFO semantic_router.utils.logger local


We run the solely static routes layer:

In [13]:
rl("Solve the equation 5-x=12 for x?")

RouteChoice(name='math', function_call=None, similarity_score=None)

In [27]:
from unify import Unify
# Environment setup for API keys
os.environ["UNIFY_KEY"] =  getpass("Enter Unify API Key: ")

In [ ]:
import os
from unify import ChatBot

def test_unify_endpoint():
    try:
        # Set up the ChatBot with a known working endpoint and API key
        agent = ChatBot(
            api_key=os.environ["UNIFY_KEY"],
            endpoint="gpt-4@anyscale"  # Adjust this if necessary
        )
        
        # Example prompt to test the service
        test_prompt = "What is 2+2?"
        
        # Making a request to the Unify service
        response = agent.run(test_prompt)
        print("Response from Unify:", response)
        
    except Exception as e:
        print("Failed to connect to Unify endpoint:", str(e))

if __name__ == "__main__":
    test_unify_endpoint()


In [30]:
#Unify process query
def process_query(query):
    route_choice = rl(query)
    print(f"Route chosen: {route_choice.name}")

    if route_choice.name == "math":
        # Initialize Unify with the endpoint for math queries
        unify = Unify(
            api_key=os.environ["UNIFY_KEY"],
            endpoint="gpt-3.5-turbo@anyscale"  # Use the correct endpoint for math queries
        )
        # Generate the response using Unify
        response = unify.generate(user_prompt=query)
        return response

    elif route_choice.name == "coding":
        # Initialize Unify with the endpoint for coding queries
        unify = Unify(
            api_key=os.environ["UNIFY_KEY"],
            endpoint="deepseek-coder-33b-instruct@anyscale"  # Use the correct endpoint for coding queries
        )
        # Generate the response using Unify
        response = unify.generate(user_prompt=query)
        return response

    else:
        return "This query does not fall under a supported category."



In [31]:
# Process query test
print(process_query("Solve the equation 5-x=12 for x?"))

Route chosen: math


InternalServerError: Error code: 500 - {'detail': 'Internal Server Error. Digest: dce6e8aa'}